# Reading the .mat files from the 5F Dataset

Add the necessary imports for preliminary steps.

In [ ]:
import pandas as pd
import numpy as np
import mne, glob
import random
import matplotlib
import pathlib
import random
from scipy.io import loadmat
from mne.io import concatenate_raws, read_raw_edf
from itertools import chain
from braindecode.datasets import create_from_X_y, BaseDataset, BaseConcatDataset
from os import listdir
from os.path import isfile, join
from braindecode.preprocessing import exponential_moving_standardize

Getting the files names of the .mat files

In [ ]:
mat_files = glob.glob("*.mat")
mat_files = [fn for fn in mat_files if "HFREQ" not in fn]
mat_files

Loads the data of the .mat files into the annots variable

In [ ]:
annots = [loadmat(fn) for fn in mat_files]

Uncomment or modify the next lines to change the subjects to train.

In [ ]:
annots = [annots[0]] # A
# annots = [annots[2]] # B
# annots = [annots[3]] # C
# annots = [annots[4]] # F
# annots = []

### Merges the .mat files into one variable.
- ```annots[i]["o"][0][0][5]``` contains the EEG signal values
- ```annots[i]["o"][0][0][4]``` contains the events

In [ ]:
merged = {
    "__header__": "Merged Files",
    "__version__": "1.0",
    "__globals__": [],
    "o": annots[-1]["o"],
}

for i in range(len(annots) - 1):
    merged["o"][0][0][5] = np.concatenate(
        (merged["o"][0][0][5], annots[i]["o"][0][0][5])
    )
    merged["o"][0][0][4] = np.concatenate(
        (merged["o"][0][0][4], annots[i]["o"][0][0][4])
    )

# Extracting events and values

Removes the unnecessary events in the events list. This leaves only classes `1`, `2`, `3`, `4`, `5`.

In [ ]:
events = [[1], [2], [3], [4], [5]]

real_events = []
real_vals = []

for i, event in enumerate(merged["o"][0][0][4]):
    if event in events:
        real_vals.append(merged["o"][0][0][5][i])
        real_events.append(merged["o"][0][0][4][i])

Flattens the list

In [ ]:
real_events = list(chain.from_iterable(real_events))

Determines the indexes where there is a change in event.

In [ ]:
left = 0
non_repeating_events = []
indexes = []

real_events.append(-1)

for right in range(1, len(real_events)):
    if real_events[right] != real_events[left]:
        indexes.append(left)  # appends the end; @ index 0, event = 1
        non_repeating_events.append(real_events[left])
    #         print(left, real_events[left])

    left += 1

Adds 0 to the start of the list.

In [ ]:
indexes = [0] + indexes
# indexes

Gets the duration of each group of events

In [ ]:
left = 0
right = 1
event_duration = []
while right < len(indexes):
    #     print(indexes[right] - indexes[left])
    event_duration.append(indexes[right] - indexes[left])
    right += 1
    left += 1

Prints the information regarding the .mat files.

In [ ]:
channel_names = [channel[0][0] for channel in merged["o"][0][0][6]]
sfreq = merged["o"][0][0][2][0][0]  # sfreq
EEG_data = merged["o"][0][0][5]
EEG_data = np.transpose(EEG_data)
n_channels, n_samples = EEG_data.shape
event_codes = merged["o"][0][0][4]
cl_lab = ["Thumb", "Index finger", "Middle finger", "Ring finger", "Pinkie finger"]

In [ ]:
print("EEG_data shape: ", EEG_data.shape)
print("Channels: ", channel_names)
print("Sampling frequency: ", sfreq)
print("Num of Samples: ", n_samples)
print("Num of Channels: ", n_channels)
print("Event codes: ", np.unique(event_codes))
print("Class names: ", cl_lab)

Concatenantes the EEG signal data and event codes. Also tranposes the values

In [ ]:
EEG_data = merged["o"][0][0][5]
EEG_concat = EEG_data
EEG_concat = np.concatenate((EEG_data, event_codes), axis=1)
EEG_concat = np.transpose(EEG_concat)

# Converting to mne.raw

Creates the info variable from mne for the `raw` object.

In [ ]:
info = mne.create_info(channel_names + ["events"], sfreq, "eeg")
info

Converts the NumPy array to mne Raw object

In [ ]:
simulated_raw = mne.io.RawArray(EEG_concat, info)

In [ ]:
simulated_raw.to_data_frame()  # Note that this multiplies the values by 1 million

# Converting to basedataset

Converts the `Raw` object to `basedataset` to preprocess the values.

In [ ]:
base_data = BaseConcatDataset([simulated_raw], target_transform=None)

In [ ]:
base_data = []
for d in [simulated_raw]:
    base_data.append(BaseDataset(d))

In [ ]:
concat_base_data = BaseConcatDataset(base_data)

In [ ]:
concat_base_data.datasets[0].raw.__dict__["_data"]

# Preprocessing

Does the necessary preprocessing to the `basedataset`. Specifically, the preprocessors used are channel reduction, bandpass filter, and exponential moving standardization.

In [ ]:
from braindecode.preprocessing import (
    exponential_moving_standardize,
    preprocess,
    Preprocessor,
    scale,
)

# original
low_cut_hz = 4.0  # low cut frequency for filtering
high_cut_hz = 38.0  # high cut frequency for filtering

# Parameters for exponential moving standardization
factor_new = 1e-3
init_block_size = 1000

preprocessors = [
    Preprocessor(
        fn="pick_channels",
        ch_names=[
            "Fp1",
            "Fp2",
            "F3",
            "F4",
            "C3",
            "C4",
            "O1",
            "O2",
            "F7",
            "F8",
            "T3",
            "T4",
            "T5",
            "T6",
        ],
    ),  # Select 14 out 22 Channels
    Preprocessor("filter", l_freq=low_cut_hz, h_freq=high_cut_hz),  # Bandpass filter
    Preprocessor("pick_types", eeg=True, meg=False, stim=False),  # Keep EEG sensors
    Preprocessor(
        exponential_moving_standardize,  # Exponential moving standardization
        factor_new=factor_new,
        init_block_size=init_block_size,
    ),
]

# Transform the data
preprocess(concat_base_data, preprocessors)

Converts the `basedataset` to a pandas `dataframe`.

In [ ]:
dataDF1 = [
    d.raw.to_data_frame(scalings=dict(eeg=1, mag=1, grad=1))
    for i, d in enumerate(concat_base_data.datasets)
]

In [ ]:
dataDF1[0]["events"] = event_codes

# Splitting Events

Only gets 256 samples for each event.

In [ ]:
DATA = []
sum = 1

for i, e in enumerate(event_duration):
    times = e // 256
    for j in range(times):
        DATA.append(dataDF1[0][sum + 256 * j : sum + 256 + 256 * j])
    sum += e

# Events

Subtracts 1 to the value of the events.
- 1 -> 0
- 2 -> 1
- 3 -> 2
- 4 -> 3
- 5 -> 4

In [ ]:
events = []
for d in DATA:
    events.append(d["events"][0:1].values[0])

In [ ]:
# Change Events, so Classes begins with 0
events_newIndex = []
for i, e in enumerate(events):
    if e == 1:
        events_newIndex.append(0)
    elif e == 2:
        events_newIndex.append(1)
    elif e == 3:
        events_newIndex.append(2)
    elif e == 4:
        events_newIndex.append(3)
    elif e == 5:
        events_newIndex.append(4)

# Create X Signal

Removes the `MarkerIndex` column.

In [ ]:
fourteen_channels = [
    "Fp1",
    "Fp2",
    "F3",
    "F4",
    "C3",
    "C4",
    "O1",
    "O2",
    "F7",
    "F8",
    "T3",
    "T4",
    "T5",
    "T6",
]

In [ ]:
dataDF1 = [
    d.raw.to_data_frame().loc[:, fourteen_channels]
    for i, d in enumerate(concat_base_data.datasets)
]

In [ ]:
data_5f = [d.loc[:, fourteen_channels] for d in DATA]

Tranposes the dataset to prepare for windowing later on.

In [ ]:
for i, d in enumerate(data_5f):
    data_5f[i] = data_5f[i].to_numpy().transpose()

# Reading our own data

Since the Emotiv EpocX produceses both `.edf` and `.csv` files that contain essential information, both files are read.
- `.edf` files contain the EEG signal values for the 14 channels.
- `.csv` files contain the event codes that correspond to certain events.

In [ ]:
current_path = pathlib.Path().resolve()  # gets current directory

dataPaths = [
    f
    for f in listdir(current_path)
    if isfile(join(current_path, f)) and f.split(".")[-1] == "edf"
]
markerFile = [
    f
    for f in listdir(current_path)
    if isfile(join(current_path, f)) and f.split(".")[-1] == "csv"
]

In [ ]:
data = [
    mne.io.read_raw_edf(path, preload=True, stim_channel="auto") for path in dataPaths
]

markers = [pd.read_csv(path) for path in markerFile]

Converts the data into a `basedataset`.

In [ ]:
base_data = BaseConcatDataset(data, target_transform=None)

In [ ]:
base_data = []
for d in data:
    base_data.append(BaseDataset(d))

In [ ]:
concat_base_data = BaseConcatDataset(base_data)

# Signal Data

Extracts the data from only the channels and store the values in `DataDF`. Also extracts the data from the `MarkerIndex` column and stores the data in `dataDF2`.

In [ ]:
data = [
    mne.io.read_raw_edf(path, preload=True, stim_channel="auto") for path in dataPaths
]

dataDF1 = [
    d.raw.to_data_frame().loc[
        :,
        [
            "AF3",
            "AF4",
            "F3",
            "F4",
            "FC5",
            "FC6",
            "O1",
            "O2",
            "F7",
            "F8",
            "T7",
            "T8",
            "P7",
            "P8",
        ],
    ]
    for i, d in enumerate(concat_base_data.datasets)
]

dataDF2 = [d.to_data_frame().loc[:, ["MarkerIndex"]] for i, d in enumerate(data)]

Preprocess the data.

In [ ]:
%%capture
preprocessed = []
for i, df in enumerate(dataDF1):
    x = df.to_numpy() - 4200
    x /= 10
    bandpassed = mne.filter.filter_data(x, 256, l_freq=4, h_freq=38)
    signal_standardized = exponential_moving_standardize(
        bandpassed, factor_new=1e-3, init_block_size=1000
    )
    preprocessed.append(signal_standardized)

Removes the rest periods, which only keeps the motor imagery parts of the dataset.

In [ ]:
dataDF = [pd.concat([dataDF1[i], dataDF2[i]], axis=1) for i, d in enumerate(data)]

In [ ]:
data_init = []
temp = []
# CONTAINS ODD NUMBERS FROM 9-21 (INCLUSIVE) AND THEIR NEGATIVE COUNTERPARTS
# THESE ARE THE MARKERS FOR THE HAND MOVEMENTS
pos_marker_vals = [i for i in range(9, 18) if i % 2 != 0]
neg_marker_vals = [i for i in range(-17, -8) if i % 2 != 0]

# print(pos_marker_vals)
# print(neg_marker_vals)

start = -1
stop = -1

# print(dataDF[0]["MarkerIndex"].unique())

for i, df in enumerate(dataDF):
    # temp = []
    for j, row in df.iterrows():
        # NEED THE .round() FUNCTION BECAUSE 15 AND 19
        # ARE 14.99999 AND 18.99999 FOR SOME REASON
        if row["MarkerIndex"].round() in pos_marker_vals:
            start = j
        elif row["MarkerIndex"].round() in neg_marker_vals:
            stop = j

        if start != -1 and stop != -1:
            data_init.append(preprocessed[i][start : stop + 1])
            start = -1
            stop = -1

Only gets the last 256 time samples of the data. The commented parts of the code block can be uncommented to retrieve more time samples.

In [ ]:
data_epoch = []

for i, df in enumerate(data_init):
    size = len(df)
    data_epoch.append(df[size - 256 : size])
#     data_epoch.append(df[size-512:size-256])
#     data_epoch.append(df[size-768:size-512])
#     data_epoch.append(df[size-1024:size-768])

Tranposes the data to prepare for windowing.

In [ ]:
for i, d in enumerate(data_epoch):
    data_epoch[i] = data_epoch[i].transpose()

# Markers

Only select the `latency`,`duration`,`marker_value`, `marker_id` columns and only select the even numbered rows (Odd numbered rows are rest states)

In [ ]:
for i, marker in enumerate(markers):
    markers[i] = marker.loc[8:20, ["latency", "duration", "marker_value", "marker_id"]][
        ::2
    ]

Appends the events to the `events` list. Uncomment the commented lines depending on how many time samples extracted earlier.

In [ ]:
events = []
tempEvents = []

for i, marker in enumerate(markers):
    tempEvents = []
    for index, row in markers[i].iterrows():
        if row["marker_value"] == "hand-thumb":
            tempEvents.append(0)
        #             tempEvents.append(0)
        #             tempEvents.append(0)
        #             tempEvents.append(0)
        elif row["marker_value"] == "hand-index":
            tempEvents.append(1)
        #             tempEvents.append(1)
        #             tempEvents.append(1)
        #             tempEvents.append(1)
        elif row["marker_value"] == "hand-middle":
            tempEvents.append(2)
        #             tempEvents.append(2)
        #             tempEvents.append(2)
        #             tempEvents.append(2)
        elif row["marker_value"] == "hand-ring":
            tempEvents.append(3)
        #             tempEvents.append(3)
        #             tempEvents.append(3)
        #             tempEvents.append(3)
        elif row["marker_value"] == "hand-pinky":
            tempEvents.append(4)
    #             tempEvents.append(4)
    #             tempEvents.append(4)
    #             tempEvents.append(4)
    events.append(tempEvents)

Flatten the list.

In [ ]:
event_list = [item for sublist in events for item in sublist]

# Merge 5F and Data from the Emotiv EpocX

In [ ]:
final_event = event_list + events_newIndex
final_data = data_epoch + data_5f

In [ ]:
print(len(final_event))
print(len(final_data))

# Windowing

In [ ]:
channel_names = [
    "AF3",
    "AF4",
    "F3",
    "F4",
    "FC5",
    "FC6",
    "O1",
    "O2",
    "F7",
    "F8",
    "T7",
    "T8",
    "P7",
    "P8",
]
sfreq = 200

In [ ]:
%%capture
windows_dataset = create_from_X_y(
    final_data,
    final_event,
    drop_last_window=False,
    sfreq=sfreq,
    ch_names=channel_names,
    window_stride_samples=256,
    window_size_samples=256,
)

In [ ]:
windows_dataset.get_metadata()

Selects the training and validation set with a 75/25 split.

In [ ]:
# Adjust baseed on Total Files
session = ["session_T" for i in range(len(final_event))]

valid_percent = round(len(final_event) * 0.25)
num_of_valids = valid_percent // 5

valids_count = [0 for i in range(5)]

s = 0
print(len(events))

while s != num_of_valids * 5:
    r = random.randint(0, len(final_event) - 1)
    if valids_count[final_event[r] - 1] != num_of_valids:
        session[r] = "session_E"
        valids_count[final_event[r] - 1] += 1
        s += 1

description = pd.DataFrame({"session": session})

In [ ]:
windows_dataset.set_description(description)  # look as dataset description

In [ ]:
windows_dataset.description

In [ ]:
splitted = windows_dataset.split("session")
train_set = splitted["session_T"]
valid_set = splitted["session_E"]

In [ ]:
# print(len(train_set))
# print(len(valid_set))

# EEGNet Training

In [ ]:
import torch
from braindecode.util import set_random_seeds
from braindecode.models import ShallowFBCSPNet
from braindecode.models import EEGNetv4

cuda = torch.cuda.is_available()  # check if GPU is available, if True chooses to use it
device = "cuda" if cuda else "cpu"
if cuda:
    torch.backends.cudnn.benchmark = True
# Set random seed to be able to roughly reproduce results
# Note that with cudnn benchmark set to True, GPU indeterminism
# may still make results substantially different between runs.
# To obtain more consistent results at the cost of increased computation time,
# you can set `cudnn_benchmark=False` in `set_random_seeds`
# or remove `torch.backends.cudnn.benchmark = True`
seed = 20200220
set_random_seeds(seed=seed, cuda=cuda)

n_classes = 5
# Extract number of chans and time steps from dataset
n_chans = train_set[0][0].shape[0]
input_window_samples = train_set[0][0].shape[1]

model = EEGNetv4(
    n_chans,
    n_classes,
    input_window_samples=input_window_samples,
    final_conv_length="auto",
    kernel_length=100,
)

# Send model to GPU
if cuda:
    model.cuda()

In [ ]:
from skorch.callbacks import LRScheduler
from skorch.helper import predefined_split
from braindecode import EEGClassifier

# These values we found good for shallow network:
lr = 0.0625 * 0.01
weight_decay = 0

batch_size = 64
n_epochs = 500

clf = EEGClassifier(
    model,
    criterion=torch.nn.NLLLoss,
    optimizer=torch.optim.AdamW,
    train_split=predefined_split(valid_set),  # using valid_set for validation
    optimizer__lr=lr,
    optimizer__weight_decay=weight_decay,
    batch_size=batch_size,
    callbacks=[
        "accuracy",
        ("lr_scheduler", LRScheduler("CosineAnnealingLR", T_max=n_epochs - 1)),
    ],
    device=device,
)

# Model training for a specified number of epochs. `y` is None as it is already supplied
# in the dataset.

In [ ]:
%%capture
clf.fit(train_set, y=None, epochs=n_epochs)

# Plotting and Matrix
### Plots the loss and missclassification rate of the training and validation.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import pandas as pd

# Extract loss and accuracy values for plotting from history object
results_columns = ["train_loss", "valid_loss", "train_accuracy", "valid_accuracy"]
df = pd.DataFrame(
    clf.history[:, results_columns],
    columns=results_columns,
    index=clf.history[:, "epoch"],
)

# get percent of misclass for better visual comparison to loss
df = df.assign(
    train_misclass=100 - 100 * df.train_accuracy,
    valid_misclass=100 - 100 * df.valid_accuracy,
)

plt.style.use("seaborn")
fig, ax1 = plt.subplots(figsize=(20, 10))
df.loc[:, ["train_loss", "valid_loss"]].plot(
    ax=ax1, style=["-", ":"], color="tab:blue", legend=False, fontsize=14
)

ax1.tick_params(axis="y", labelcolor="tab:blue", labelsize=14)
ax1.set_ylabel("Loss", color="tab:blue", fontsize=14)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

df.loc[:, ["train_misclass", "valid_misclass"]].plot(
    ax=ax2, style=["-", ":"], color="tab:red", legend=False
)
ax2.tick_params(axis="y", labelcolor="tab:red", labelsize=14)
ax2.set_ylabel("Misclassification Rate [%]", color="tab:red", fontsize=14)
ax2.set_ylim(ax2.get_ylim()[0], 85)  # make some room for legend
ax1.set_xlabel("Epoch", fontsize=14)

# where some data has already been plotted to ax
handles = []
handles.append(
    Line2D([0], [0], color="black", linewidth=1, linestyle="-", label="Train")
)
handles.append(
    Line2D([0], [0], color="black", linewidth=1, linestyle=":", label="Valid")
)
plt.legend(handles, [h.get_label() for h in handles], fontsize=14)
plt.tight_layout()

### Generates the confusion matrix.

In [ ]:
from sklearn.metrics import confusion_matrix
from braindecode.visualization import plot_confusion_matrix

# generate confusion matrices
# get the targets
y_true = valid_set.get_metadata().target
y_pred = clf.predict(valid_set)

# generating confusion matrix
confusion_mat = confusion_matrix(y_true, y_pred)

# add class labels
# label_dict is class_name : str -> i_class : int
# label_dict = valid_set.datasets[0].windows.event_id.items()

label_dict = {
    "Thumb": 0,
    "Index": 1,
    "Middle": 2,
    "Ring": 3,
    "Pinky": 4,
}.items()

# sort the labels by values (values are integer class labels)
labels = list(dict(sorted(list(label_dict), key=lambda kv: kv[1])).keys())

# plot the basic conf. matrix
plot_confusion_matrix(confusion_mat, class_names=labels, with_f1_score=True)

# Saving the model
Uncomment to save your model.

In [ ]:
# import torch
# torch.save(model.state_dict(), "file_name.pt")